In [12]:
import torch
from torchvision import datasets,transforms
from torch.utils.data import DataLoader





In [13]:
trainset_plain=datasets.CIFAR10(root='./data',train=True,download=True,transform=transforms.ToTensor())
loader=DataLoader(trainset_plain,batch_size=50000,shuffle=False,num_workers=2)
data_iter=iter(loader)
images,labels=next(data_iter)
mean=images.mean(dim=[0,2,3])
std=images.std(dim=[0,2,3])
print('mean',mean,'std',std)

mean tensor([0.4914, 0.4822, 0.4465]) std tensor([0.2470, 0.2435, 0.2616])


In [14]:
train_transform=transforms.Compose([
    transforms.RandomCrop(32,padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean,std)
])

test_transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean,std)
])

trainset=datasets.CIFAR10(root='./data',train=True,download=False,transform=train_transform)
testset=datasets.CIFAR10(root='./data',train=False,download=True,transform=test_transform)

train_loader=DataLoader(trainset,batch_size=128,shuffle=True,num_workers=2)
test_loader=DataLoader(testset,batch_size=128,shuffle=False,num_workers=2)

print(len(trainset),len(testset))


50000 10000


In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PatchEmbed(nn.Module):
    def __init__(self,img_size=32,patch_size=4,in_chans=3,embed_dim=192):
        super().__init__()
        self.img_size=img_size
        self.patch_size=patch_size
        self.embed_dim=embed_dim
        self.grid_size=img_size//patch_size
        self.num_patches=self.grid_size*self.grid_size
        self.proj=nn.Conv2d(in_chans,embed_dim,patch_size,patch_size)
    def forward(self,x):
        x=self.proj(x)
        x=x.flatten(2).transpose(1,2) #[batch, seq_len=64, embed_dim=192] transformer expects this
        return x

class ResidualConnection(nn.Module):
    def __init__(self,fn):
        super().__init__()
        self.fn=fn
    def forward(self,x):
        return x+self.fn(x)

class MHSA(nn.Module):
    def __init__(self,embed_dim,num_heads=12,attn_dropout=0.1,proj_dropout=0.1): #according to wu et al
        super().__init__()
        self.num_heads=num_heads #12
        self.head_dim=embed_dim//num_heads
        self.scale=self.head_dim**-0.5
        self.qkv=nn.Linear(embed_dim,embed_dim*3) # qkv at a time
        self.attn_drop=nn.Dropout(attn_dropout)
        self.proj=nn.Linear(embed_dim,embed_dim)
        self.proj_drop=nn.Dropout(proj_dropout)
    def forward(self,x):
        B,N,C=x.shape
        qkv=self.qkv(x).reshape(B,N,3,self.num_heads,self.head_dim).permute(2,0,3,1,4) #(B, N, 3*C) to (B, N, 3, num_heads, head_dim) to (3, B, num_heads, N, head_dim)

        q,k,v=qkv[0],qkv[1],qkv[2]
        attn=(q@k.transpose(-2,-1))*self.scale
        attn=attn.softmax(dim=-1)
        attn=self.attn_drop(attn)
        x=(attn@v).transpose(1,2).reshape(B,N,C) # back into embedding space
        x=self.proj(x)
        x=self.proj_drop(x)
        return x

class MLP(nn.Module):
    def __init__(self,embed_dim,mlp_ratio=4,drop=0.1):
        super().__init__()
        self.fc1=nn.Linear(embed_dim,embed_dim*mlp_ratio)
        self.fc2=nn.Linear(embed_dim*mlp_ratio,embed_dim)
        self.act=nn.GELU()
        self.drop=nn.Dropout(drop)
    def forward(self,x):
        x=self.fc1(x)
        x=self.act(x)
        x=self.drop(x)
        x=self.fc2(x)
        x=self.drop(x) # each dropout call in independent
        return x

class Block(nn.Module):
    def __init__(self,embed_dim,num_heads=12,mlp_ratio=4,drop=0.1):
        super().__init__()
        self.norm1=nn.LayerNorm(embed_dim)
        self.attn=MHSA(embed_dim,num_heads,drop,drop)
        self.res1=ResidualConnection(lambda x:self.attn(self.norm1(x)))
        self.norm2=nn.LayerNorm(embed_dim)
        self.mlp=MLP(embed_dim,mlp_ratio,drop)
        self.res2=ResidualConnection(lambda x:self.mlp(self.norm2(x)))
    def forward(self,x):
        x=self.res1(x)
        x=self.res2(x)
        return x

class ViT(nn.Module):
    def __init__(self,img_size=32,patch_size=4,in_chans=3,embed_dim=192,num_classes=10,depth=12,num_heads=12,mlp_ratio=4,drop=0.1):
        super().__init__()
        self.patch_embed=PatchEmbed(img_size,patch_size,in_chans,embed_dim)
        self.cls_token = nn.Parameter(torch.empty(1,1,embed_dim))
        nn.init.trunc_normal_(self.cls_token, mean=0.0, std=0.02) # as implemented in the ViT paper

        self.pos_embed = nn.Parameter(torch.empty(1, 1+self.patch_embed.num_patches, embed_dim))
        nn.init.trunc_normal_(self.pos_embed, std=0.02)

        self.pos_drop=nn.Dropout(drop)
        self.blocks=nn.ModuleList([Block(embed_dim,num_heads,mlp_ratio,drop) for _ in range(depth)])
        self.norm=nn.LayerNorm(embed_dim)
        self.head=nn.Linear(embed_dim,num_classes)
    def forward(self,x):
        B=x.shape[0]
        x=self.patch_embed(x)
        cls_tokens=self.cls_token.expand(B,-1,-1) #(B, 1, embed_dim)
        x=torch.cat((cls_tokens,x),dim=1)
        x=x+self.pos_embed
        x=self.pos_drop(x)
        for blk in self.blocks:
            x=blk(x)
        x=self.norm(x)
        cls_out=x[:,0]
        x=self.head(cls_out)
        return x

model=ViT()
sample_images, _=next(iter(train_loader))
sample_images=sample_images[:2]
out=model(sample_images)
print(out.shape)


torch.Size([2, 10])


In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm


device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = ViT().to(device)
scaler = torch.amp.GradScaler()

class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self,smoothing=0.1):
        super().__init__()
        self.smoothing=smoothing
        self.confidence=1.0-smoothing
    def forward(self,pred,target):
        logprobs=nn.functional.log_softmax(pred,dim=-1)
        nll_loss=-logprobs.gather(dim=-1,index=target.unsqueeze(1)).squeeze(1)
        smooth_loss=-logprobs.mean(dim=-1)
        loss=self.confidence*nll_loss+self.smoothing*smooth_loss
        return loss.mean()

criterion=LabelSmoothingCrossEntropy(0.1)
optimizer=optim.AdamW(model.parameters(),lr=3e-4,weight_decay=0.05)
T_max=200
scheduler=optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=T_max)
warmup_epochs=10
initial_lr=3e-5

epochs=50

for epoch in range(epochs):
    model.train()
    loop=tqdm(train_loader)
    running_loss=0
    correct=0
    total=0
    if epoch<warmup_epochs:
        lr=initial_lr+(3e-4-initial_lr)*(epoch/warmup_epochs)
        for g in optimizer.param_groups:
            g['lr']=lr
    for imgs,labels in loop:
        imgs,labels=imgs.to(device),labels.to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            outputs=model(imgs)
            loss=criterion(outputs,labels)
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
        scaler.step(optimizer)
        scaler.update()
        running_loss+=loss.item()*imgs.size(0)
        _,preds=outputs.max(1)
        correct+=preds.eq(labels).sum().item()
        total+=labels.size(0)
        loop.set_description(f"Epoch [{epoch+1}/{epochs}]")
        loop.set_postfix(loss=running_loss/total,acc=100*correct/total)
    if epoch>=warmup_epochs:
        scheduler.step()
    model.eval()
    test_loss=0
    correct=0
    total=0
    with torch.no_grad():
        for imgs,labels in test_loader:
            imgs,labels=imgs.to(device),labels.to(device)
            with torch.cuda.amp.autocast():
                outputs=model(imgs)
                loss=criterion(outputs,labels)
            test_loss+=loss.item()*imgs.size(0)
            _,preds=outputs.max(1)
            correct+=preds.eq(labels).sum().item()
            total+=labels.size(0)
    print(f"Test Loss:{test_loss/total:.4f} Test Acc:{100*correct/total:.2f}%")


  0%|          | 0/391 [00:00<?, ?it/s]/tmp/ipython-input-3981880474.py:45: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch [1/50]: 100%|██████████| 391/391 [00:44<00:00,  8.87it/s, acc=29, loss=1.99]
/tmp/ipython-input-3981880474.py:68: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Test Loss:1.9496 Test Acc:33.19%


Epoch [2/50]: 100%|██████████| 391/391 [00:43<00:00,  9.07it/s, acc=35.4, loss=1.86]


Test Loss:1.9301 Test Acc:33.97%


Epoch [3/50]: 100%|██████████| 391/391 [00:43<00:00,  8.96it/s, acc=41.2, loss=1.75]


...

Test Loss:0.9242 Test Acc:82.56%


Epoch [49/50]: 100%|██████████| 391/391 [00:44<00:00,  8.79it/s, acc=88.9, loss=0.747]


Test Loss:0.9354 Test Acc:82.38%


Epoch [50/50]: 100%|██████████| 391/391 [00:44<00:00,  8.81it/s, acc=89, loss=0.745]


Test Loss:0.9368 Test Acc:82.45%
